In [1]:
from glob import glob

import numpy as np
import pandas as pd
import joblib
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from utilities import *

In [17]:
random_state = 42
code_folder_path = ""
data_percentage = 0.3 # how much of the data we will use

In [18]:
names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
#     "Gaussian Process",
    "Decision Tree",
    "Random Forest",
#     "Neural Net", # Too slow
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=100),
#     MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

In [19]:
df = get_SMADC_folder_data()
df = df.sample(frac=data_percentage)

In [20]:
%%time
X_train, X_test, y_train, y_test = train_test_split(df["Text"], df["Region"], random_state=random_state)
count_vectorizer = CountVectorizer()
X_vectorized = count_vectorizer.fit_transform(X_train)
# standard_scaler = StandardScaler(with_mean=False).fit(X_vectorized)

Wall time: 5.32 s


In [21]:
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [ ]:
%%time
for name, clf in zip(names, classifiers):
    if name in ("Naive Bayes", "Gaussian Process", "QDA"):
        model = make_pipeline(count_vectorizer, DenseTransformer(), clf)
    else:
        model = make_pipeline(count_vectorizer, clf)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(name, score, flush=True)
    joblib.dump(model, join(code_folder_path, f"models/other_models/{name}_acc={score.round(3)}.model"))

Nearest Neighbors 0.5686467553367729
